In [1]:
import pandas as pd

# 加载Excel文件
xls = pd.ExcelFile("E:\\数学建模国赛2023选题\\C题\\第二题建模约束参数.xlsx")


In [2]:
# 查看Excel文件中各个sheet的前几行数据以了解结构和列名
sheet_names = xls.sheet_names

# 存储每个sheet的前几行数据
sheet_previews = {}

for sheet in sheet_names:
    sheet_previews[sheet] = pd.read_excel(xls, sheet).head()

sheet_previews


{'Model_Params':            a         b   星期   分类名称
 0  84.277492  0.974480  星期三  水生根茎类
 1  81.092421  1.009424  星期三    花叶类
 2  14.822394  1.011291  星期三    花菜类
 3  25.241518  0.999187  星期三     茄类
 4  61.474034  1.000100  星期三    辣椒类,
 'Markup_Bounds':     分类名称  lower_bound  upper_bound
 0  水生根茎类    44.059830    96.064736
 1    花叶类    55.054544    97.460568
 2    花菜类    36.620075    83.754887
 3     茄类    43.469963   134.360511
 4    辣椒类    46.272717   148.951008,
 'Sales_Bounds':     分类名称  lower_bound  upper_bound
 0  水生根茎类      4.82050     91.51175
 1    花叶类    105.64975    289.32900
 2    花菜类      8.09800     79.35975
 3     茄类     12.86075     51.80800
 4    辣椒类     33.14950    151.92225,
 'Daily_Sales_Bounds':    lower_bound  upper_bound
 0    228.75775    650.33475}

In [3]:
model_params = pd.read_excel(xls, 'Model_Params')

# 读取成本加价范围约束
markup_bounds = pd.read_excel(xls, 'Markup_Bounds')

# 读取销量范围约束
sales_bounds = pd.read_excel(xls, 'Sales_Bounds')

# 读取每日销售总量约束
daily_sales_bounds = pd.read_excel(xls, 'Daily_Sales_Bounds')

In [4]:
!pip install pyswarm

In [5]:
import numpy as np
from pyswarm import pso

# 定义目标函数
def objective(X):
    x = X[:42]
    y = X[42:]
    return -np.sum(x * y)

# 从Excel读取的模型参数和约束条件
a_values = model_params['a'].values
b_values = model_params['b'].values

markup_lower_bound = markup_bounds['lower_bound'].values
markup_upper_bound = markup_bounds['upper_bound'].values

sales_lower_bound = sales_bounds['lower_bound'].values
sales_upper_bound = sales_bounds['upper_bound'].values

daily_sales_lower_bound = daily_sales_bounds['lower_bound'].values[0]
daily_sales_upper_bound = daily_sales_bounds['upper_bound'].values[0]

In [6]:
# 使用NumPy的round函数，设置小数点后3位
a_values_rounded = np.round(a_values, 3)
b_values_rounded = np.round(b_values, 3)
markup_lower_bound_rounded = np.round(markup_lower_bound, 3)
markup_upper_bound_rounded = np.round(markup_upper_bound, 3)
sales_lower_bound_rounded = np.round(sales_lower_bound, 3)
sales_upper_bound_rounded = np.round(sales_upper_bound, 3)
daily_sales_lower_bound_rounded = np.round(daily_sales_lower_bound, 3)
daily_sales_upper_bound_rounded = np.round(daily_sales_upper_bound, 3)

a_values_rounded, b_values_rounded, markup_lower_bound_rounded, markup_upper_bound_rounded, sales_lower_bound_rounded, sales_upper_bound_rounded, daily_sales_lower_bound_rounded, daily_sales_upper_bound_rounded


(array([ 84.277,  81.092,  14.822,  25.242,  61.474,  37.374,  71.976,
        151.796,  11.087,  26.986,  40.648,  38.869,  39.525,  77.623,
          4.725,  27.216,  32.784,  35.459, 128.708, 159.5  ,  14.82 ,
         45.058,  67.346,  47.863,  22.573, 133.643,   9.753,  34.378,
         66.992,  55.238, 103.486,  78.828,  13.512,  28.868,  54.179,
         38.839,  76.971, 169.65 ,   9.583,  22.7  ,  46.647,  27.557]),
 array([0.974, 1.009, 1.011, 0.999, 1.   , 1.   , 0.975, 1.001, 1.018,
        0.999, 1.006, 1.   , 0.987, 1.01 , 1.032, 1.   , 1.009, 1.002,
        0.974, 1.004, 1.018, 0.996, 1.003, 1.002, 1.002, 1.006, 1.025,
        1.   , 1.003, 1.   , 0.972, 1.01 , 1.013, 0.999, 1.001, 1.   ,
        0.977, 0.999, 1.02 , 1.   , 1.003, 1.004]),
 array([44.06 , 55.055, 36.62 , 43.47 , 46.273, 45.747]),
 array([ 96.065,  97.461,  83.755, 134.361, 148.951,  89.779]),
 array([  4.82 , 105.65 ,   8.098,  12.861,  33.15 ,  24.312]),
 array([ 91.512, 289.329,  79.36 ,  51.808, 151.92

In [10]:
# 定义约束函数
def constraints(X):
    # 初始化约束列表
    con = []
    
    # 42个模型软约束
    epsilon = 0.1  # 选择一个合适的epsilon值
    x = X[:42]
    y = X[42:]
    for i in range(42):
        con.append(abs(y[i] - a_values[i] * np.exp(b_values[i] * x[i])) - epsilon)

    
    
    # 12个成本加价范围约束
    for i in range(6):
        con.append(markup_lower_bound[i] - np.min(x[i*7:i*7+7]))
        con.append(np.max(x[i*7:i*7+7]) - markup_upper_bound[i])
        
    # 12个销量范围约束
    for i in range(6):
        con.append(sales_lower_bound[i] - np.min(y[i*7:i*7+7]))
        con.append(np.max(y[i*7:i*7+7]) - sales_upper_bound[i])
        
    # 每日销售总量的上下界约束
    con.append(daily_sales_upper_bound - np.sum(y))
    con.append(np.sum(y) - daily_sales_lower_bound)
    
    return con

# 参数范围：x和y都在0-100之间（可根据实际情况调整）
lb = [0]*84
ub = [100]*84

# 使用PSO算法进行优化
xopt, fopt = pso(objective, lb, ub, f_ieqcons=constraints)

xopt, fopt
 

Stopping search: maximum iterations reached --> 100
However, the optimization couldn't find a feasible design. Sorry


(array([71.67293983, 72.66106745, 98.78826457,  6.63756658, 97.5438113 ,
         2.38511962,  5.69528123, 98.57215494, 34.41511982, 14.81563134,
        90.92095504, 72.1389436 , 76.07606455, 81.54772742, 22.99083434,
        90.68441584,  2.52262133, 59.48606238, 80.35221397, 15.62550165,
        41.03447229, 37.09502591, 79.96136596, 22.44048986, 87.41753641,
        38.20105835, 15.74688953, 81.40806608, 78.84156297,  2.93722227,
         4.23538891, 95.38732575, 14.91598739, 73.62413513, 17.8972844 ,
        39.40604249, 43.63320906, 62.24220746, 12.03670615, 86.55030871,
        59.15365668, 46.31885067, 34.94816503, 89.76714344, 39.37190366,
        15.44706813, 14.50762329, 23.9296024 , 43.10452346, 92.5672462 ,
        87.23517394, 97.90104026, 42.36676042, 88.59464887, 86.68685453,
        35.55757167, 99.30125618, 36.03163592,  5.48710712, 52.25463808,
        27.64406116, 60.02057596, 21.6745658 , 49.17615746, 10.13320177,
        36.99658726, 69.68414625, 99.18804435, 73.8